# Keep artifacts local in a cloud instance

If you want to default to keeping artifacts local in a cloud instance, enable {attr}`~lamindb.setup.core.InstanceSettings.keep_artifacts_local`.

In [ ]:
!lamin login testuser1
!lamin init --storage s3://lamindb-ci/keep-artifacts-local

In [ ]:
# !pip install 'lamindb[jupyter,aws]'
import lamindb as ln

ln.track("l9lFf83aPwRc0000")

In [ ]:
# the setting should be enabled on lamin.ai
# we're temporarily setting it here only for testing purposes
ln.setup.settings.instance._keep_artifacts_local = True

You can register a managed local storage location as follows:

In [ ]:
ln.settings.storage_local = "./my_storage_local"

Now, you have two storage locations: one in the S3 bucket, and the other locally.

In [ ]:
ln.Storage.df()

## Update storage description

You can add a description to the storage by using the description parameter:

In [ ]:
storage_record = ln.Storage.get(root=ln.settings.storage_local.root)
storage_record.description = "Files stored locally in site X on server Y for reason ABC"
storage_record.save()
ln.Storage.df()

## Use local storage

If you save an artifact, by default, it's stored in local storage.

In [ ]:
original_filepath = ln.core.datasets.file_fcs()
artifact = ln.Artifact(original_filepath, description="My fcs file").save()
local_path = artifact.path
local_path

You'll see the `.fcs` file named by the `uid` in your `.lamindb/` directory under `./my_storage_local/`:

In [ ]:
ln.settings.storage_local.root.view_tree()

In [ ]:
assert local_path.exists()
assert artifact.path.as_posix().startswith(ln.settings.storage_local.root.as_posix())

If you'd like to upload an artifact, you pass `upload=True` to the `save()` method.

In [ ]:
artifact.save(upload=True)

You now see the artifact in the S3 bucket:

In [ ]:
ln.setup.settings.storage.root.view_tree()

And it's no longer present in local storage:

In [ ]:
ln.setup.settings.instance.storage_local.root.view_tree()

In [ ]:
assert artifact.path.exists()
assert not local_path.exists()
assert artifact.path.as_posix().startswith(
    ln.setup.settings.instance.storage.root.as_posix()
)

## Direct upload

You can also directly upload a file by passing `upload=True`:

In [ ]:
filepath = ln.core.datasets.file_mini_csv()
artifact2 = ln.Artifact(filepath, description="My csv file").save(upload=True)
artifact2.path

Now we have two files on S3:

In [ ]:
ln.Artifact.df(include="storage__root")

In [ ]:
assert artifact2.path.exists()

## Pre-existing artifacts

Assume we already have a file in our registered local storage location:

In [ ]:
file_in_local_storage = ln.core.datasets.file_bam()
file_in_local_storage.rename("./my_storage_local/output.bam")
ln.UPath("my_storage_local/").view_tree()

If we create an artifact from it, it remains where it is during saving:

In [ ]:
my_existing_file = ln.Artifact(
    "./my_storage_local/output.bam", description="my existing file"
).save()
ln.UPath("my_storage_local/").view_tree()

The storage path of the artifact is constructed using `key` because  `key_is_virtual=False`:

In [ ]:
my_existing_file

However, if we decide to upload the artifact, we'll use the `uid` for constructing the storage path and switch `key_is_virtual=True`:

In [ ]:
my_existing_file.save(upload=True)

Here is the remote path of the artifact:

In [ ]:
my_existing_file.path

And here are the contents of the storage locations:

In [ ]:
# the path on S3
ln.setup.settings.storage.root.view_tree()
# the local path
ln.setup.settings.instance.storage_local.root.view_tree()

## Delete the test instance

Delete the artifacts:

In [ ]:
artifact.delete(permanent=True)
artifact2.delete(permanent=True)
my_existing_file.delete(permanent=True)

Delete the instance:

In [ ]:
ln.setup.delete("keep-artifacts-local", force=True)